Import stuff and initialize global parameters.

In [8]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Conv2D, Flatten, Dense, LSTM, \
    MaxPooling2D
from keras.models import Sequential
from common import extract_training_data

IMAGE_SIZE = (64, 64, 3)

# LSTM network
Reading data and creating initial model.

In [9]:
images, labels = extract_training_data("Data/20171029-201949.h264.avi", "Data/20171029-201949.h264.csv", IMAGE_SIZE)

In [10]:
y_train = labels.as_matrix(columns=labels.columns[1: -1])
y_train.shape

(1744, 3)

In [11]:
images.shape

(1744, 64, 64, 3)

In [ ]:
from keras.layers import Permute, Reshape

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), kernel_initializer="he_normal", activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Permute((3, 2, 1)))
    model.add(Reshape((4, 7688)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(3, activation='linear'))

    model.compile(loss="mse", optimizer="adam")
    
    return model

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
permute_2 (Permute)          (None, 32, 31, 31)        0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 7688)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                1984768   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 1,985,859
Trainable params: 1,985,859
Non-trainable params: 0
_________________________________________________________________


### LSTM network results

In [ ]:
history = model.fit(images, y_train, batch_size=64, epochs=10, validation_split=0.04)

Train on 1674 samples, validate on 70 samples
Epoch 1/10
1674/1674 [==============================] - 15s - loss: 0.4226 - val_loss: 0.4023
Epoch 2/10
1674/1674 [==============================] - 13s - loss: 0.1045 - val_loss: 0.2867
Epoch 3/10
1674/1674 [==============================] - 13s - loss: 0.0888 - val_loss: 0.2952
Epoch 4/10
1674/1674 [==============================] - 12s - loss: 0.0771 - val_loss: 0.2901
Epoch 5/10
1674/1674 [==============================] - 14s - loss: 0.0720 - val_loss: 0.2897
Epoch 6/10
1600/1674 [===========================>..] - ETA: 0s - loss: 0.0676

In [ ]:
test_images, test_labels = extract_training_data("Data/20171029-201639.h264.avi", 
                                                            "Data/20171029-201639.h264.csv", 
                                                            IMAGE_SIZE)

prediction = model.predict(test_images)

In [ ]:
prediction

In [ ]:
plt.hist(prediction[:, 0])
plt.show()

In [ ]:
plt.hist(prediction[:, 1])
plt.show()

In [ ]:
plt.hist(prediction[:, 2])
plt.show()

In [ ]:
validation_labels = test_labels.as_matrix(columns=test_labels.columns[1: -1])
clipped_prediction = np.zeros_like(prediction)

clipped_prediction[:, 0] = np.clip(prediction[:, 0], -1, 1)
clipped_prediction[:, 1] = np.clip(prediction[:, 1], 0, 1)
clipped_prediction[:, 2] = np.clip(prediction[:, 2], 0, 1)

print(validation_labels.shape)
print(clipped_prediction.shape)

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(validation_labels, prediction)

## K fold cross validation

Currently the model is just a copy of what is above, defined as a function for the KerasRegressor wrapper

In [ ]:
from common import run_kfold_cross_val

# Concatenate both videos
total_images = np.concatenate((images, test_images), axis=0)
total_labels = np.concatenate((labels, test_labels), axis=0)

total_y_train = total_labels[:, 1:-1]

results = run_kfold_cross_val(create_model, total_images, total_y_train)
print(results)
print(results.mean())
print(np.median(results))

## Saving model
Saves the model into a file and loads it again.

In [ ]:
from keras.models import load_model

model.save('lstm.h5')
del model
model = load_model('lstm.h5')